# Description

Calculates the Bayesian credibility for the hypothesis that the event occurrence rate during a test B is higher than that during a test A, given the test duration and number of observed events for both test run A and B.

# Parameters

In [1]:
test_duration_A = 100
n_events_A = 20

test_duration_B = 10
n_events_B = 1

round_to_digits = 4

assert isinstance(test_duration_A, (int, float)) and test_duration_A > 0
assert isinstance(test_duration_B, (int, float)) and test_duration_B > 0
assert isinstance(n_events_A, int) and n_events_A >= 0
assert isinstance(n_events_B, int) and n_events_B >= 0
assert isinstance(round_to_digits, int) and round_to_digits >= 1

# Evaluation

In [ ]:
import matplotlib.pyplot as plt
import mpmath as mp
import numpy as np
from scipy import integrate

mp.mp.dps = 50 #use 50 decimal digits precision in mpmath calculations

#posterior is unnormalized to save calculation time, remember to normalize result when needed
def posterior_prob_poisson_r(r_event, test_duration, n_events, prior):
    return mp.power(mp.mpf(r_event*test_duration), n_events) / mp.fac(n_events) * mp.exp(-r_event*test_duration) * prior(r_event, test_duration)

posterior_prob_poisson_r_vectorized = np.vectorize(posterior_prob_poisson_r)

def posterior_pairs(r_A, test_duration_A, n_events_A, prior_A, r_B, test_duration_B, n_events_B, prior_B):
    return posterior_prob_poisson_r_vectorized(r_A, test_duration_A, n_events_A, prior=prior_A) * posterior_prob_poisson_r_vectorized(r_B, test_duration_B, n_events_B, prior=prior_B)

#split the integral into constant and non-constant parts and prescale so numerical integration does not fail due to sharp peak for large test_duration
def integrate_posterior_prob_poisson_r(test_duration, n_events, prior, bounds):
    mean = (n_events+1) / test_duration
    std_dev = (n_events+1)**0.5 / test_duration
    prescale = 1/posterior_prob_poisson_r(mean, test_duration, n_events, prior)
    prior_prescaled = lambda r, duration: prior(r, duration) * prescale
    
    result = 0
    if bounds[0] < mean-3*std_dev:
        result += integrate.quad(posterior_prob_poisson_r, max(0, bounds[0]), min(mean-3*std_dev, bounds[1]), args=(test_duration, n_events, prior_prescaled))[0]
    if bounds[0] < mean and bounds[1] >= mean-3*std_dev:
        result += integrate.quad(posterior_prob_poisson_r, max(mean-3*std_dev, bounds[0]), min(mean, bounds[1]), args=(test_duration, n_events, prior_prescaled))[0]
    if bounds[0] < mean+3*std_dev and bounds[1] >= mean:
        result += integrate.quad(posterior_prob_poisson_r, max(mean, bounds[0]), min(mean+3*std_dev, bounds[1]), args=(test_duration, n_events, prior_prescaled))[0]
    if bounds[1] >= mean+3*std_dev:
        result += integrate.quad(posterior_prob_poisson_r, max(mean+3*std_dev, bounds[0]), bounds[1], args=(test_duration, n_events, prior_prescaled))[0]
        
    return result / prescale
    
#split the integral into constant and non-constant parts and prescale so numerical integration does not fail due to sharp peak for large n_trial
def integrate_lower_triangle(test_duration_A, n_events_A, prior_A, test_duration_B, n_events_B, prior_B):
    mean_A = (n_events_A+1) / test_duration_A
    std_dev_A = (n_events_A+1)**0.5 / test_duration_A
    prescale_A = 1/posterior_prob_poisson_r(mean_A, test_duration_A, n_events_A, prior_A)
    prior_prescaled_A = lambda r, duration: prior_A(r, duration) * prescale_A
    
    mean_B = (n_events_B+1) / test_duration_B
    std_dev_B = (n_events_B+1)**0.5 / test_duration_B
    prescale_B = 1/posterior_prob_poisson_r(mean_B, test_duration_B, n_events_B, prior_B)
    prior_prescaled_B = lambda r, duration: prior_B(r, duration) * prescale_B

    bounds = [0, np.inf]

    f = lambda x: posterior_prob_poisson_r(x, test_duration_A, n_events_A, prior_prescaled_A) * integrate_posterior_prob_poisson_r(test_duration_B, n_events_B, prior_prescaled_B, [0, x])
    
    result = 0
    if bounds[0] < mean_A-3*std_dev_A:
        result += integrate.quad(f, max(0, bounds[0]), min(mean_A-3*std_dev_A, bounds[1]))[0]
    if bounds[0] < mean_A and bounds[1] >= mean_A-3*std_dev_A:
        result += integrate.quad(f, max(mean_A-3*std_dev_A, bounds[0]), min(mean_A, bounds[1]))[0]
    if bounds[0] < mean_A+3*std_dev_A and bounds[1] >= mean_A:
        result += integrate.quad(f, max(mean_A, bounds[0]), min(mean_A+3*std_dev_A, bounds[1]))[0]
    if bounds[1] >= mean_A+3*std_dev_A:
        result += integrate.quad(f, max(mean_A+3*std_dev_A, bounds[0]), bounds[1])[0]
        
    return result / prescale_A / prescale_B

r_plotmax_A = (n_events_A+1)/test_duration_A + 3*(n_events_A+1)**0.5/test_duration_A #mean+5*std_dev
r_plotmax_B = (n_events_B+1)/test_duration_B + 3*(n_events_B+1)**0.5/test_duration_B #mean+5*std_dev
r_maxplot = max(r_plotmax_A, r_plotmax_B)

r_A = np.linspace(0, r_maxplot, 101)
r_B = np.linspace(0, r_maxplot, 101)

prior_A = lambda r, duration: 1
prior_B = lambda r, duration: 1

norm_A = integrate_posterior_prob_poisson_r(test_duration_A, n_events_A, prior_A, [0, np.inf])
norm_B = integrate_posterior_prob_poisson_r(test_duration_B, n_events_B, prior_B, [0, np.inf])

X, Y = np.meshgrid(r_A, r_B)
posterior_probs = posterior_pairs(X, test_duration_A, n_events_A, prior_A, Y, test_duration_B, n_events_B, prior_B) / (norm_A * norm_B)

P_rA_larger_rB = integrate_lower_triangle(test_duration_A, n_events_A, prior_A, test_duration_B, n_events_B, prior_B) / (norm_A * norm_B)
P_rB_larger_rA = integrate_lower_triangle(test_duration_B, n_events_B, prior_B, test_duration_A, n_events_A, prior_A) / (norm_A * norm_B)
P_rA_larger_rB_rounded = mp.nstr(mp.mpf(P_rA_larger_rB), round_to_digits)
P_rB_larger_rA_rounded = mp.nstr(mp.mpf(P_rB_larger_rA), round_to_digits)
P_sum = P_rB_larger_rA + P_rA_larger_rB
numerical_error_estimate = abs(P_sum - mp.mpf(1))

plt.contourf(X, Y, posterior_probs, cmap='Blues')
plt.axline([0,0], [r_maxplot,r_maxplot], color='red', alpha=0.3)

plt.gca().set_title('Poisson experiment {} events / duration {} (A) vs. {} events / duration {} (B)\nBayesian pair posterior with uniform prior\nOrder of numerical error: {}%'.format(n_events_A,
                    test_duration_A, n_events_B, test_duration_B, mp.nstr(numerical_error_estimate * 100, round_to_digits)))
plt.gca().set_xlabel('event occurrence rate A')
plt.gca().set_ylabel('event occurence rate B')
plt.gca().tick_params(axis='x', which='minor', bottom=True)
plt.gca().minorticks_on()
plt.grid(which='major')
plt.grid(which='minor', alpha=0.3);
plt.gca().set_aspect('equal')

plt.text(r_maxplot*0.3, r_maxplot*0.7, '$P(r_B > r_A)$\n=${}$'.format(P_rB_larger_rA_rounded), horizontalalignment='center', verticalalignment='center', fontsize='xx-large', fontweight='bold', alpha=0.6)
plt.text(r_maxplot*0.7, r_maxplot*0.3, '$P(r_A > r_B)$\n=${}$'.format(P_rA_larger_rB_rounded), horizontalalignment='center', verticalalignment='center', fontsize='xx-large', fontweight='bold', alpha=0.6);